Este notebook ayuda a probar el modelo **LLM** entrenado para clasificar atacantes en análisis de tráfico de red. Para usar el modelo es necesario un entorno con mínimo **50 GB de RAM**, es posible exportar los resultados en un fichero **JSON** o llamar directamente a la **API** del **SIEM** **Wazuh** para almacenar los resultados a modo de eventos.

Si se usa en la plataforma Google Colab, es útil montar Google Drive y crear un directorio en el que trabajar, en todo caso deben de consultarse las rutas utilizadas en este ejemplo y adaptarse.



*Ivan Dunlap, 2024*

## PASO 1: instalación de dependencias necesarias en el entorno
Para ejecutar este notebook, es necesario instalar varias dependencias. A continuación, se muestra el código para instalar estas dependencias en el entorno de ejecución.

In [ ]:
%cd /content/drive/MyDrive/PASIR/model
!pip install torch torchvision torchaudio
!pip install datasets
!pip install transformers
!pip install aggregate
!pip install accelerate
!pip install flash_attn
!pip install argparse
!pip install -e git+https://github.com/OpenAccess-AI-Collective/axolotl#egg=axolotl
!pip install huggingface_hub==0.23.3

## PASO 2: autenticación en Hugging Face (ver sección obtención de token)

Para utilizar modelos de la plataforma Hugging Face, es necesario obtener un token y logearse.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## PASO 3: clonar los repositorios necesarios (en caso de no tenerlos ya)

Si ya se han clonado anteriormente y se dispone de ellos en el directorio actual, este paso no es necesario.

In [ ]:
!git clone https://github.com/dhd2000/DissertationFiles.git
!git clone https://github.com/KayvanKarim/ntfa.git
!git clone https://github.com/OpenAccess-AI-Collective/axolotl.git
!git clone https://github.com/DunlapIvan-Clase/PASIR.git

## PASO 4: procesar los datos para que sean aceptados por el modelo
Este paso es importante para que los datos provenientes tanto de un dataset como el utilizado para entrenar el modelo, Intrusion detection evaluation dataset (CIC-IDS2017) como los datos recogidos por nuestro colector, sean utilzables por el mismo.

Es necesario cambiar la ruta y el fichero csv que queramos utilizar para aplicar la transformación.

A la hora de ejecutar el script ntfa, si no disponemos de un fichero de configuración json, debemos de configurar el índice de cada columna de nuestro dataset para que el script la reconozca.

In [ ]:
import os

# Directorio donde se encuentra el archivo CSV
traffic_dir = r'./CIDDS-001/CIDDS-001/traffic/OpenStack'

# Nombre del archivo específico a utilizar
target_csv_file = 'CIDDS-001-internal-week1.csv'

target_csv_path = os.path.join(traffic_dir, target_csv_file)

!python3 ntfa/ntfa.py {target_csv_path}
!python3 DissertationFiles/convert.py output.csv
!python3 DissertationFiles/balance_dataset.py output.jsonl

## PASO 5: utilizar el modelo mediante el script.

Una vez que tengamos los datos, podemos utilizar el script para probar el modelo o para utilizarlo en un entorno real.

### Ejemplos de uso del script

#### Ejemplo 1: enviar los resultados del modelo a Wazuh mediante la API (ideal para ejecuciones locales)

!python3 ../PASIR/traffic_analyzer/traffic_analyzer.py send ruta-fichero-jsonl-generado usuario-api-wazuh pass-api-wazuh host

#### Ejemplo 2: almacenar los resultados del modelo en un fichero formato "JSON"
!python3 ../PASIR/traffic_analyzer/traffic_analyzer.py save ruta-fichero-jsonl-generado ruta-fichero-json-salida


#### Ejemplo 3: enviar los datos del fichero "JSON" a Wazuh mediante la API
!python3 ../PASIR/traffic_analyzer/traffic_analyzer.py sendfile  ruta-fichero-json usuario-api-wazuh pass-api-wazuh host




In [ ]:
%cd /content/drive/MyDrive/PASIR/model/mistral-NIDS
!python3 ../PASIR/traffic_analyzer/traffic_analyzer.py save /content/drive/MyDrive/PASIR/model/output_test.jsonl /content/drive/MyDrive/PASIR/model/traffic_analyzer/output2.json

## PASO 6: utilizar el modelo para crear una estadística de su funcionamiento.

Para comprobar el funcionamiento del modelo y crear una estadística de como está funcionando, podemos utilizar este paso, aunque es necesario que los datos que utilicemos se encuentren etiquetados, como los del dataset de entrenamiento.

In [ ]:
import torch
torch.cuda.empty_cache()
%cd /content/drive/MyDrive/PASIR/model/mistral-NIDS
!cp /content/drive/MyDrive/PASIR/model/DissertationFiles/finetune_test_llama.py ./finetune_test.py
!sed -i "s/NousResearch\/Llama\-2\-7b\-hf/mistralai\/Mistral\-7B\-v0\.1/" ./finetune_test.py # Cambiar modelo base

!python3 finetune_test.py /content/drive/MyDrive/PASIR/model/output.jsonl